# **Modul 5: EDA _(Exploratory Data Analysis)_ Analisa Deskriptif dan Pengelompokkan Data**


## **1. Impor _packages_ atau modul yang dibutuhkan**


In [ ]:
# Mengecek pada sistem operasi apa dan dimana direktori bekerja
import os 
os.getcwd()

In [2]:
# Mengimpor packages yang dibutuhkan
import pandas as pd
import numpy as np

## **2. Impor Data dari CSV ke DataFrame**


In [3]:
# Mengimpor data dari file csv ke dalam variabel yang memuat dataframe dengan pandas
df_csv = pd.read_csv('Online Retail Data.csv', header=0)
df_csv.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN


## **3. Pembersihan dan Pentransformasian Data**


### **3.1. Mengindeks ulang urutan kolom hingga memfilter data yang hilang**


In [4]:
# Menduplikasi dataframe untuk dibersihkan
df1 = df_csv.copy()

# Re-index urutan kolom
df_reindex = df1.reindex(
  columns=['order_id','order_date','customer_id',
           'product_code','product_name','quantity','price']
)

# Menstandarisasikan data pada produk kolom
df_reindex['product_name'] = df_reindex['product_name'].str.lower()

# Memfilter data berdasarkan na value
df_notna = df_reindex[df_reindex.notna().all(axis=1)].copy()

# Membuat kolom baru untuk order status berdasarkan order_id untuk canceled and delivered
df_notna['order_status'] = np.where(
  df_notna['order_id'].str[:1]=='C', 'cancelled', 'delivered'
)

df_notna.head()

,order_id,order_date,customer_id,product_code,product_name,quantity,price,order_status
0,493410,2010-01-04 09:24:00,12346.0,TEST001,this is a test product.,5,4.50,delivered
1,C493411,2010-01-04 09:43:00,14590.0,21539,retro spots butter dish,-1,4.25,cancelled
2,493412,2010-01-04 09:53:00,12346.0,TEST001,this is a test product.,5,4.50,delivered
6,493414,2010-01-04 10:28:00,14590.0,21844,retro spot mug,36,2.55,delivered
7,493414,2010-01-04 10:28:00,14590.0,21533,retro spot large milk jug,12,4.25,delivered


### **3.2. Memfilter produk dengan mengecualikan produk tes**


In [5]:
# Pola dalam product_name dan product_code untuk test
product_code = r'^TEST\d+$'
product_name = r'^This is\d+$'

# Memfilter data berdasarkan nilai yang tidak kosong 
df_nottest = df_notna[(
  ~df_notna['product_code'].str.match(product_code, case=False, na=False)
)]

# Memfilter data dengan mengecualikan order id untuk cancelled
df_nottest.head()

,order_id,order_date,customer_id,product_code,product_name,quantity,price,order_status
1,C493411,2010-01-04 09:43:00,14590.0,21539,retro spots butter dish,-1,4.25,cancelled
6,493414,2010-01-04 10:28:00,14590.0,21844,retro spot mug,36,2.55,delivered
7,493414,2010-01-04 10:28:00,14590.0,21533,retro spot large milk jug,12,4.25,delivered
8,493414,2010-01-04 10:28:00,14590.0,37508,new england ceramic cake server,2,2.55,delivered
9,493414,2010-01-04 10:28:00,14590.0,35001G,hand open shape gold,2,4.25,delivered


### **3.3. Menormalisai nilai kuantitas yang negatif karena sudah diberikan label order_status**


In [6]:
# Mengubah nilai quantity yang negatif karena sudah diberikan label status order cancelled
df_notnegative = df_nottest.copy()
df_notnegative['quantity'] = df_notnegative['quantity'].abs()
df_notnegative.head()


,order_id,order_date,customer_id,product_code,product_name,quantity,price,order_status
1,C493411,2010-01-04 09:43:00,14590.0,21539,retro spots butter dish,1,4.25,cancelled
6,493414,2010-01-04 10:28:00,14590.0,21844,retro spot mug,36,2.55,delivered
7,493414,2010-01-04 10:28:00,14590.0,21533,retro spot large milk jug,12,4.25,delivered
8,493414,2010-01-04 10:28:00,14590.0,37508,new england ceramic cake server,2,2.55,delivered
9,493414,2010-01-04 10:28:00,14590.0,35001G,hand open shape gold,2,4.25,delivered


### **3.4. Memfilter untuk nilai price hanya untuk nilai yang positif**


In [7]:
# Memfilter data dengan nilai yang positif
df_price = df_notnegative.copy()
df_price = df_price[df_price['price'] > 0]
df_price.head()

,order_id,order_date,customer_id,product_code,product_name,quantity,price,order_status
1,C493411,2010-01-04 09:43:00,14590.0,21539,retro spots butter dish,1,4.25,cancelled
6,493414,2010-01-04 10:28:00,14590.0,21844,retro spot mug,36,2.55,delivered
7,493414,2010-01-04 10:28:00,14590.0,21533,retro spot large milk jug,12,4.25,delivered
8,493414,2010-01-04 10:28:00,14590.0,37508,new england ceramic cake server,2,2.55,delivered
9,493414,2010-01-04 10:28:00,14590.0,35001G,hand open shape gold,2,4.25,delivered


### **3.5. Membuat jumlah total pembelian**


In [8]:
# Jumlah total pembelian
df_price['total_amt'] = (
  df_price['quantity'] 
  * df_price['price']
)
df_price.head()

,order_id,order_date,customer_id,product_code,product_name,quantity,price,order_status,total_amt
1,C493411,2010-01-04 09:43:00,14590.0,21539,retro spots butter dish,1,4.25,cancelled,4.25
6,493414,2010-01-04 10:28:00,14590.0,21844,retro spot mug,36,2.55,delivered,91.80
7,493414,2010-01-04 10:28:00,14590.0,21533,retro spot large milk jug,12,4.25,delivered,51.00
8,493414,2010-01-04 10:28:00,14590.0,37508,new england ceramic cake server,2,2.55,delivered,5.10
9,493414,2010-01-04 10:28:00,14590.0,35001G,hand open shape gold,2,4.25,delivered,8.50


### **3.5. Menstandarisasi Produk Name**


In [9]:
# Menstandarisasi product_name dengan mencari frekuensi
most_freq_pn = (
  df_price
  .groupby(['product_code','product_name'], as_index=False)
  .agg(order_count=('order_id','nunique'))
  .sort_values(['product_code','order_count'], ascending=[True,False])
)
most_freq_pn['rank'] = most_freq_pn.groupby('product_code')['order_count'].rank(method='first', ascending=False)
most_freq_pn = most_freq_pn[most_freq_pn['rank']==1].drop(columns=['order_count','rank'])
most_freq_pn.head()

,product_code,product_name
0,10002,inflatable political globe
1,10080,groovy cactus inflatable
2,10120,doggy rubber
3,10123C,hearts wrapping tape
4,10123G,army camo wrapping tape


In [10]:
# Melakukan merge untuk standarisasi dari product_name
df_clean = (
    df_price
    .merge(most_freq_pn.rename(columns={'product_name': 'most_freq_product_name'}), how='left', on='product_code')
)

df_clean['product_name']=df_clean['most_freq_product_name']
df_clean=df_clean.drop(columns='most_freq_product_name')
df_clean.head()

,order_id,order_date,customer_id,product_code,product_name,quantity,price,order_status,total_amt
0,C493411,2010-01-04 09:43:00,14590.0,21539,red retrospot butter dish,1,4.25,cancelled,4.25
1,493414,2010-01-04 10:28:00,14590.0,21844,red retrospot mug,36,2.55,delivered,91.80
2,493414,2010-01-04 10:28:00,14590.0,21533,retro spot large milk jug,12,4.25,delivered,51.00
3,493414,2010-01-04 10:28:00,14590.0,37508,new england ceramic cake server,2,2.55,delivered,5.10
4,493414,2010-01-04 10:28:00,14590.0,35001G,hand open shape gold,2,4.25,delivered,8.50


### **3.6. Mentransformasi tipe data**


In [11]:
# Mengecek tipe data
print("Data Type Sebelum:\n")
df_clean.info()

Data Type Sebelum:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360828 entries, 0 to 360827
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      360828 non-null  object 
 1   order_date    360828 non-null  object 
 2   customer_id   360828 non-null  float64
 3   product_code  360828 non-null  object 
 4   product_name  360828 non-null  object 
 5   quantity      360828 non-null  int64  
 6   price         360828 non-null  float64
 7   order_status  360828 non-null  object 
 8   total_amt     360828 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 24.8+ MB


In [12]:
# Mentransformasi tipe data
df_clean['order_id']=df_clean['order_id'].astype(str)
df_clean['order_date']= pd.to_datetime(df_clean['order_date'], format='ISO8601', utc=False)
df_clean['customer_id']=df_clean['customer_id'].astype('Int64')
df_clean['product_code']=df_clean['product_code'].astype(str)
df_clean['product_name']=df_clean['product_name'].astype(str)
df_clean['quantity']=df_clean['quantity'].astype(int)
df_clean['order_status']=df_clean['order_status'].astype(str)

print("\nData Type Sesudah:\n")
df_clean.info()


Data Type Sesudah:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360828 entries, 0 to 360827
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      360828 non-null  object        
 1   order_date    360828 non-null  datetime64[ns]
 2   customer_id   360828 non-null  Int64         
 3   product_code  360828 non-null  object        
 4   product_name  360828 non-null  object        
 5   quantity      360828 non-null  int32         
 6   price         360828 non-null  float64       
 7   order_status  360828 non-null  object        
 8   total_amt     360828 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 23.7+ MB


## **4. Exploratory Data Analysis (EDA): Statistik Deskriptif dan Pengelompokkan Data**


Hal yang biasa dilakukan dalam Exploratory Data Analysis:

1. Ringkasan Statistik dari Data seperti:
   1. Menampilkan jumlah banyaknya baris dari catatan data (_records_).
   2. Menampilkan tipe data dari dataframe pada setiap kolom.
   3. Menampilkan Statistik Deskriptif seperti mean, median atau modus, variansi, standar deviasi, n-tile, nilai minimum dan maximum.
   4. Mendapatkan statistik dari variabel tertentu (kolom) seperti jumlah, nilai unik, jarak interval nilai terbesar dan nilai terkecil (_range_) dan statistik dasar lainnya.
2. Menangani Variabel dengan data kategori/kelompok
3. Visualisasi Data
4. Analisis Data:
   1. Analisis Korelasi
   2. Analisis Rangkaian Waktu (_TimeSeries Analysis_)
   3. Analisis menggunakan pembelajaran mesin (_Machine Learning_), dsb.

Pada tahapan Modul 5 ini EDA dilakukan sampai tahap ke-2, Visualisasi Data akan dibahas pada Modul 6.


### **4.1. Mendapatkan Statistik Deskriptif dari Data**


#### **4.1.1. Menampilkan banyaknya jumlah dari catatan data (records)**

> Menampilkan banyaknya jumlah dari catatan data dengan fungsi <code>DataFrame.shape</code>.
>
> <code>DataFrame.shape</code> mengembalikan _tuple_ yang merepresentasikan dimensi dari DataFrame, memberikan nilai dari jumlah baris dan kolom

Kode dibawah merupakan kode untuk menampilkan output yang disesuaikan (_customize_)


In [13]:
num_rows, num_columns= df_clean.shape
print(f"Number of Rows:\n{num_rows}")
print(f"Number of Columns:\n{num_columns}")

Number of Rows:
360828
Number of Columns:
9


#### **4.1.2. Menampilkan tipe data dari setiap kolom**

> Menampilkan tipe data dari setiap kolom dengan menggunakan fungsi <code>DataFrame.info()</code>
> Fungsi tersebut menilai struktur data dan mengidentifikasi isu potensial dari DataFrame (nilai yang hilang dan tipe data)


In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360828 entries, 0 to 360827
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      360828 non-null  object        
 1   order_date    360828 non-null  datetime64[ns]
 2   customer_id   360828 non-null  Int64         
 3   product_code  360828 non-null  object        
 4   product_name  360828 non-null  object        
 5   quantity      360828 non-null  int32         
 6   price         360828 non-null  float64       
 7   order_status  360828 non-null  object        
 8   total_amt     360828 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 23.7+ MB


> Mendapatkan tipedata dari setiap kolom dengan menggunakkan fungsi <code>DataFrame.dtypes</code>


In [15]:
df_clean.dtypes

order_id                object
order_date      datetime64[ns]
customer_id              Int64
product_code            object
product_name            object
quantity                 int32
price                  float64
order_status            object
total_amt              float64
dtype: object

#### **4.1.3. Mendapatkan Statistik Deskriptif dari DataFrame**

> Menampilkan statistik dasar dari data dengan nilai integer dapat menggunakan <code>DataFrame.describe(percentiles=None, include=None, exclude=None)</code>


In [16]:
df_clean.describe().round(2)

,order_date,customer_id,quantity,price,total_amt
count,360828,360828.0,360828.00,360828.00,360828.00
mean,2010-07-22 05:55:24.100734720,15557.69,11.95,3.67,20.47
min,2010-01-04 09:43:00,12346.0,1.00,0.00,0.00
25%,2010-04-25 11:43:00,14210.0,2.00,1.25,4.25
50%,2010-08-02 14:24:00,15580.0,4.00,1.95,10.40
75%,2010-10-24 16:03:00,16938.0,12.00,3.75,17.70
max,2010-12-23 16:06:00,18287.0,10000.00,25111.09,25111.09
std,NaN,1593.68,68.51,70.38,97.74


Pandas memberlakukan kolom yang berisi tipe data datetime memiliki semantik yang kaya melebihi nilai numerik sederhana. Jika Anda memerlukan representasi numerik tertentu, Anda harus menangani konversi tersebut secara eksplisit.


#### **4.1.4. Mendapatkan Statistik pada Kolom Tertentu dari DataFrame**

##### a. Menampilkan Statistik Dasar seperti menghitung untuk Variabel (kolom) berdasarkan nilai unik


In [17]:
unique_product_code = pd.DataFrame({'unique_product_code':df_clean['product_code'].unique()})
unique_product_name = pd.DataFrame({'unique_product_name':df_clean['product_name'].unique()})
unique_product_code

,unique_product_code
0,21539
1,21844
2,21533
3,37508
4,35001G
...,...
3892,90136
3893,22970
3894,22858
3895,22971


In [18]:
unique_product_name

,unique_product_name
0,red retrospot butter dish
1,red retrospot mug
2,retro spot large milk jug
3,new england ceramic cake server
4,hand open shape gold
...,...
3863,pale pink/amethyst stone necklace
3864,london bus coffee mug
3865,easter tin keepsake
3866,queens guard coffee mug


Menampikan jumlah total order dan jumlah total kuantitas untuk masing-masing produk
dengan mengikuti rumus untuk nilai natural:

> $$[\text{Sum} = \sum_{i=1}^{n} x_i]$$


In [19]:
product_order_count = (
  df_clean
  .groupby('product_name',observed=True)
  .agg(total_quantity=pd.NamedAgg(column='quantity',aggfunc='sum'),
        total_amt=pd.NamedAgg(column='total_amt',aggfunc='sum'))
  .sort_values(by='total_quantity',ascending=False)
)
product_order_count = product_order_count.reset_index()
product_order_count.rename(columns={'total_quantity': 'sum_total_quantity','total_amt': 'sum_total_amt'},inplace=True)
product_order_count

,product_name,sum_total_quantity,sum_total_amt
0,world war 2 gliders asstd designs,50038,10373.66
1,white hanging heart t-light holder,49204,131331.95
2,brocade ring purse,48031,8707.31
3,pack of 72 retro spot cake cases,42796,20511.09
4,jumbo bag red retrospot,40988,71852.40
...,...,...,...
3863,turq pendant triple shell necklace,1,4.95
3864,pearl and cherry quartz braclet,1,3.50
3865,diamante bow brooch white colour,1,6.35
3866,diamante necklace black,1,7.50


##### b. Mencari Statistik Deskriptif Dasar untuk variabel tertentu (kolom)


Mencari mean atau rerata menggunakan fungsi <code>.mean()</code> untuk variabel, atau berdasarkan suatu kolom pada Pandas mengikuti rumus:

Dengan anggapan $m$ baris dan $n$ jumlah kolom, untuk mengkalkulasi setiap kolom (axis=0)

> $$[\text{Mean}j = \frac{1}{m} \sum{i=1}^{m} x_{ij}]$$

Keterangan:
$(x_{ij})$ merepresentasikan nilai jumlah pada baris ($i$) dan kolom ($j$)


In [20]:
mean_price=df_clean['price'].mean()
mean_total_amt=df_clean['total_amt'].mean()
mean_quantity=df_clean['quantity'].mean()

mean_df_clean = pd.DataFrame(
    {'mean': [mean_price, mean_total_amt, mean_quantity]},
    index=['price', 'total_amt', 'quantity']
)
mean_df_clean['mean']= mean_df_clean['mean'].round(2)
mean_df_clean

,mean
price,3.67
total_amt,20.47
quantity,11.95


Mencari modus dengan menggunakan fungsi <code>.mode()</code> untuk variabel, atau kolom pada Pandas mengikuti rumus:

Dengan anggapan _m_ baris dan _n_ kolom. Untuk kolom _j_ modus-nya adalah:

> $$[\text{Mode}_j = \text{value with highest frequency in column } j ]$$

Jika terdapat banyak nilai dengan frekuensi tertinggi yang sama, semuanya akan dipertimbangkan sebagai modus.

menambahkan <code>.iloc[0]</code> mengekstrak nilai yang dibutuhkan, angka 0 memilih elemen awal pada seri, yang sesuai dengan nilai modus dan jika terdapat banyak modus akan memberikan nilai modus paling awal yang ditemui.


In [21]:
mode_price=df_clean['price'].mode().iloc[0]
mode_total_amt=df_clean['total_amt'].mode().iloc[0]
mode_quantity=df_clean['quantity'].mode().iloc[0]

mode_df_clean = pd.DataFrame(
    {'mode': [mode_price, mode_total_amt, mode_quantity]}, index=['price', 'total_amt', 'quantity']
)
mode_df_clean

,mode
price,1.25
total_amt,15.00
quantity,1.00


Mencari median dengan menggunakan fungsi <code>.median()</code> untuk variabel, atau kolom mengikuti rumus:

- Jika jumlah observasi atau $n$ merupakan nilai ganjil maka median akan berada tepat di tengah nilai

  > $$[\text{Median} = \left(\frac{n + 1}{2}\right)\text{th term}]$$

- Jika jumlah obeservasi atau $n$ merupakan nilai genap maka median merupakan rerata dari dua nilai di tengah
  > $$[\text{Median} = \frac{\left(\frac{n}{2}\right)\text{th term} + \left(\frac{n}{2} + 1\right)\text{th term}}{2}]$$


In [22]:
median_price=df_clean['price'].median()
median_total_amt=df_clean['total_amt'].median()
median_quantity=df_clean['quantity'].median()

median_df_clean = pd.DataFrame(
    {'median': [median_price, median_total_amt, median_quantity]}, 
    index=['price', 'total_amt', 'quantity']
)
median_df_clean

,median
price,1.95
total_amt,10.40
quantity,4.00


<code>.var()</code> pada Pandas untuk mencari nilai varians dari sampel pada setiap kolomnya mengikuti rumus:

> $$[ s^2 = \frac{\sum_{i=1}^{n} (x_i - \bar{x})^2}{n-1} ]$$

Keterangan:

- $s^2$: Merepresentasikan variansi dari sampel.
- $x_i$: Merepresentasikan nilai individu dari sampel.
- $\bar{x}$: mean (rerata) dari sample.
- $n$: ukuran sampel.


In [23]:
quantity_variance = df_clean['quantity'].var()
price_variance = df_clean['price'].var()
total_amt_variance = df_clean['total_amt'].var()

variance_df_clean = pd.DataFrame(
    {'Variance': [price_variance, total_amt_variance, quantity_variance]},
    index=['price', 'total_amt', 'quantity']
)
variance_df_clean['Variance'] = variance_df_clean['Variance'].round(2)
variance_df_clean

,Variance
price,4953.48
total_amt,9552.36
quantity,4693.02


Untuk mencari standar deviasi menggunakan pandas dapat menggunakan <code>.std()</code>, kalkulasi yang digunakan mengikuti rumus:

1. Standar deviasi untuk populasi pada pandas menggunakan <code>.std(ddof=0)</code>:
   > $$ [\sigma = \sqrt{\frac{\sum_{i=1}^{N} (x_i - \mu)^2}{N}}] $$

Keterangan:

- $N$ merepresentasikan total dari poin data pada suatu populasi.
- $x_i$ merepresentasikan nilai individual dari dataset.
- $\mu$ merupakan mean dari populasi (average).

Bagian dari formula mengartikan:

- $\sum_{i=1}^{N}$ menunjukan penjumlahan dari selisih kuadrat antara setiap titik data $(x_i)$ dan mean $(\mu)$
- $(x_i - \mu)^2$ menghitung deviasi kuadrat setiap titik data dari mean
- Seluruh ekspresi dibagi dengan $(N)$ untuk menghitung deviasi kuadrat rata-rata.
- Tahap terakhir adalah mengambil akar kuadrat dari rata-rata ini untuk mendapatkan deviasi standar populasi.

2. Standar deviasi untuk sampel pada pandas menggunakan <code>.std(ddof=1)</code>:
   > $$[s = \sqrt{\frac{\sum_{i=1}^{n} (x_i - \bar{x})^2}{n-1}}]$$

Keterangan:

- $(n)$ merepresentasikan ukuran sampel.
- $(\bar{x})$ adalah mean dari sampel.
- Penggunaan $n-1$ merupakan "_koreksi Bessel_" karena mengoreksi bias (menuju nilai yang lebih rendah) dalam estimasi varians yang diperkenalkan ketika mean sampel dari a digunakan sebagi pengganti mean yang sebenarnya dari suatu populasi.


In [24]:
quantity_std_s = df_clean['quantity'].std(ddof=1)
price_std_s = df_clean['price'].std(ddof=1)
total_amt_std_s = df_clean['total_amt'].std(ddof=1)

quantity_std_p = df_clean['quantity'].std(ddof=0)
price_std_p = df_clean['price'].std(ddof=0)
total_amt_std_p = df_clean['total_amt'].std(ddof=0)

std_df_clean = pd.DataFrame({
  'Metric':['Quantity','Price','Total Amount'],
  'std_sample':[quantity_std_s,price_std_s,total_amt_std_s],
  'std_population':[quantity_std_p,price_std_p,total_amt_std_p]
})
std_df_clean['std_sample'] = std_df_clean['std_sample'].round(2)
std_df_clean['std_population'] = std_df_clean['std_population'].round(2)
std_df_clean

,Metric,std_sample,std_population
0,Quantity,68.51,68.51
1,Price,70.38,70.38
2,Total Amount,97.74,97.74


Mencari nilai Minimum dan nilai Maximum dari suatu kolom menggunakan <code>.max()</code> dan <code>min()</code>, pandas akan mencari nilai terendah dan tertinggi secara otomatis.


In [25]:
max_quantity = df_clean['quantity'].max()
max_price = df_clean['price'].max()
max_total_amt = df_clean['total_amt'].max()

min_quantity = df_clean['quantity'].min()
min_price = df_clean['price'].min()
min_total_amt = df_clean['total_amt'].min()

df_clean_min_max = pd.DataFrame({
  'Metric':['Quantity','Price','Total Amount'],
  'max':[max_quantity,max_price,max_total_amt],
  'min':[min_quantity,min_quantity,min_total_amt]
})
df_clean_min_max

,Metric,max,min
0,Quantity,10000.00,1.000
1,Price,25111.09,1.000
2,Total Amount,25111.09,0.001


Mencari nilai dari _Kuartil_ untuk _Kuartil 1 (Q1)_ atau dikenal dengan kuartile bawah, _Kuratil 2 (Q2)_ nilainya akan sama dengan median dan _Kuartile 3 (Q3)_ atau dengan kuartile atas dengan menggunakan fungsi <code>.quantile()</code>.

1. **Kuartil 1 (Q1)** atau persentil ke-25, mewakili batas nilai bawah dimana 25% dari data berada. Pada pandas menggunakan fungsi <code>.quantile(0.25)</code>:

   > $$[Q1 = \left(\frac{n+1}{4}\right)\text{th Term}]$$

2. **Kuartil 2 (Q2)** atau persentil ke-50 dan dapat juga dikatakan sebagai median merupakan nilai tengah dari data yang ada. Pada pandas menggunakan fungsi <code>.quantile(0.50)</code>:

   > $$[Q2 = \left(\frac{n+1}{2}\right)\text{th Term}]$$

3. **Kuartil 3 (Q3)** atau persentil ke-75 merupakan batas nilai atas dimana 75% dari data berada.Pada pandas menggunakan fungsi <code>.quantile(0.75)</code>:

   > $$[Q3 = \left(\frac{3(n+1)}{4}\right)\text{th Term}]$$

   Keterangan:

- $n$ mewakili jumlah total data (observasi).


In [26]:
price_quartiles =df_clean['price'].quantile([0.25,0.50,0.75])
total_amt_quartiles =df_clean['total_amt'].quantile([0.25,0.50,0.75])
quantity_quartiles=df_clean['quantity'].quantile([0.25,0.50,0.75])

quartile_df = pd.DataFrame({
    'Metric': ['quantity', 'price', 'total_amt'],
    'Q1 (25th percentile)': [quantity_quartiles[0.25],price_quartiles[0.25],total_amt_quartiles[0.25]],
    'Median (50th percentile)': [quantity_quartiles[0.50],price_quartiles[0.50],total_amt_quartiles[0.50]],
    'Q3 (75th percentile)': [quantity_quartiles[0.75],price_quartiles[0.75],total_amt_quartiles[0.75]]
})
quartile_df

,Metric,Q1 (25th percentile),Median (50th percentile),Q3 (75th percentile)
0,quantity,2.00,4.00,12.00
1,price,1.25,1.95,3.75
2,total_amt,4.25,10.40,17.70


Mencari nilai _Jangkauan Interquartile (IQR)_ mengikuti rumus:

> $$[\text{IQR} = Q3 - Q1]$$

Dengan penjelasan:

- $(Q1)$ kuartil pertama (persentil ke-25).
- $(Q3)$ kuartil ke-tiga (persentil ke-75).


In [27]:
price_quartiles = df_clean['price'].quantile([0.25,0.50,0.75])
total_amt_quartiles = df_clean['total_amt'].quantile([0.25,0.50,0.75])
quantity_quartiles= df_clean['quantity'].quantile([0.25,0.50,0.75])

price_iqr = price_quartiles[0.75] - price_quartiles[0.25]
total_amt_iqr = total_amt_quartiles[0.75] - total_amt_quartiles[0.25]
quantity_iqr = quantity_quartiles[0.75] - quantity_quartiles[0.25]

interquartile_df = pd.DataFrame({
    'Metric': ['quantity', 'price', 'total_amt'],
    'Q1 (25th percentile)': [quantity_quartiles[0.25],price_quartiles[0.25],total_amt_quartiles[0.25]],
    'Median (50th percentile)': [quantity_quartiles[0.50],price_quartiles[0.50],total_amt_quartiles[0.50]],
    'Q3 (75th percentile)': [quantity_quartiles[0.75],price_quartiles[0.75],total_amt_quartiles[0.75]],
    'IQR':[quantity_iqr,price_iqr,total_amt_iqr]
})
interquartile_df

,Metric,Q1 (25th percentile),Median (50th percentile),Q3 (75th percentile),IQR
0,quantity,2.00,4.00,12.00,10.00
1,price,1.25,1.95,3.75,2.50
2,total_amt,4.25,10.40,17.70,13.45


> Membuat fungsi baru untuk mencari IQR:


In [28]:
iqr_values = {}
for col in df_clean.select_dtypes(include='number'):
    q1 = df_clean[col].quantile(0.25)
    q3 = df_clean[col].quantile(0.75)
    iqr_values[col] = q3 - q1
    
iqr_value_df = pd.DataFrame(iqr_values, index=['IQR'])
iqr_value_df

,customer_id,quantity,price,total_amt
IQR,2728.0,10.0,2.5,13.45


In [29]:
columns_to_analyze = ['quantity','price','total_amt']

iqr_values = {}
for col in columns_to_analyze:
    q1 = df_clean[col].quantile(0.25)
    q3 = df_clean[col].quantile(0.75)
    iqr_values[col] = q3 - q1
    
iqr_value_df1 = pd.DataFrame(iqr_values, index=['IQR'])
iqr_value_df1

,quantity,price,total_amt
IQR,10.0,2.5,13.45


> Menyatukan semua-nya menjadi satu tabel output:


In [30]:
mean_quantity=df_clean['quantity'].mean()
mean_price=df_clean['price'].mean()
mean_total_amt=df_clean['total_amt'].mean()

mode_quantity=df_clean['quantity'].mode().iloc[0]
mode_price=df_clean['price'].mode().iloc[0]
mode_total_amt=df_clean['total_amt'].mode().iloc[0]

median_quantity=df_clean['quantity'].median()
median_price=df_clean['price'].median()
median_total_amt=df_clean['total_amt'].median()

quantity_variance = df_clean['quantity'].var()
price_variance = df_clean['price'].var()
total_amt_variance = df_clean['total_amt'].var()

quantity_std_s = df_clean['quantity'].std(ddof=1)
price_std_s = df_clean['price'].std(ddof=1)
total_amt_std_s = df_clean['total_amt'].std(ddof=1)

quantity_std_p = df_clean['quantity'].std(ddof=0)
price_std_p = df_clean['price'].std(ddof=0)
total_amt_std_p = df_clean['total_amt'].std(ddof=0)

min_quantity = df_clean['quantity'].min()
min_price = df_clean['price'].min()
min_total_amt = df_clean['total_amt'].min()

max_quantity = df_clean['quantity'].max()
max_price = df_clean['price'].max()
max_total_amt = df_clean['total_amt'].max()

quantity_range = df_clean['quantity'].max() - df_clean['quantity'].min()
price_range = df_clean['price'].max() - df_clean['price'].min()
total_amt_range = df_clean['total_amt'].max() - df_clean['total_amt'].min()

quantity_iqr = quantity_quartiles[0.75] - quantity_quartiles[0.25]
price_iqr = price_quartiles[0.75] - price_quartiles[0.25]
total_amt_iqr = total_amt_quartiles[0.75] - total_amt_quartiles[0.25]

df_clean_summary = pd.DataFrame({
  'Metric':['Quantity','Price','Total Amount'],
  'mean':[mean_quantity, mean_price, mean_total_amt],
  'mode':[mode_quantity,mode_price,mode_total_amt],
  'median': [median_quantity,median_price,median_total_amt],
  'variance':[quantity_variance,price_variance,total_amt_variance],
  'std_sample':[quantity_std_s,price_std_s,total_amt_std_s],
  'std_population':[quantity_std_p,price_std_p,total_amt_std_p],
  'min':[min_quantity,min_price,min_total_amt],
  'Q1 (25th percentile)': [quantity_quartiles[0.25],price_quartiles[0.25],total_amt_quartiles[0.25]],
  'Median (50th percentile)': [quantity_quartiles[0.50],price_quartiles[0.50],total_amt_quartiles[0.50]],
  'Q3 (75th percentile)': [quantity_quartiles[0.75],price_quartiles[0.75],total_amt_quartiles[0.75]],
  'max':[max_quantity,max_price,max_total_amt],
  'range':[quantity_range,price_range,total_amt_range],
  'IQR':[quantity_iqr,price_iqr,total_amt_iqr]
})

df_clean_summary['mean']= df_clean_summary['mean'].round(2)
df_clean_summary['variance'] = df_clean_summary['variance'].round(2)
df_clean_summary['std_sample'] = df_clean_summary['std_sample'].round(2)
df_clean_summary['std_population'] = df_clean_summary['std_population'].round(2)

df_clean_summary

,Metric,mean,mode,median,variance,std_sample,std_population,min,Q1 (25th percentile),Median (50th percentile),Q3 (75th percentile),max,range,IQR
0,Quantity,11.95,1.00,4.00,4693.02,68.51,68.51,1.000,2.00,4.00,12.00,10000.00,9999.000,10.00
1,Price,3.67,1.25,1.95,4953.48,70.38,70.38,0.001,1.25,1.95,3.75,25111.09,25111.089,2.50
2,Total Amount,20.47,15.00,10.40,9552.36,97.74,97.74,0.001,4.25,10.40,17.70,25111.09,25111.089,13.45


> Mengubah urutan (_Transpose_):


In [31]:
df_clean_summary_transposed = df_clean_summary.T
df_clean_summary_transposed.columns = df_clean_summary_transposed.iloc[0]
df_clean_summary_transposed = df_clean_summary_transposed[1:]
df_clean_summary_transposed

Metric,Quantity,Price,Total Amount
mean,11.95,3.67,20.47
mode,1.0,1.25,15.0
median,4.0,1.95,10.4
variance,4693.02,4953.48,9552.36
std_sample,68.51,70.38,97.74
std_population,68.51,70.38,97.74
min,1.0,0.001,0.001
Q1 (25th percentile),2.0,1.25,4.25
Median (50th percentile),4.0,1.95,10.4
Q3 (75th percentile),12.0,3.75,17.7


> Menggunakan _loop_ secara iteratif dari kode diatas untuk mendapatkan code yang lebih ringkas dan efisien:


In [32]:
# Menentukan kolom yang akan dihitung
columns_of_interest = ['quantity', 'price', 'total_amt']

# Menyediakan kamus (dictionary) untuk menyimpan ringkasan statistik
mean_dict, mode_dict, median_dict, variance_dict = {}, {}, {}, {}
std_sample_dict, std_population_dict, min_dict, max_dict = {}, {}, {}, {}
q1_dict, q2_dict, q3_dict, iqr_dict = {}, {}, {}, {}
range_dict = {}

# Mengkalkulasi ringkasan statistik untuk setiap kolom
for col in columns_of_interest:
    mean_dict[col] = df_clean[col].mean()
    mode_dict[col] = df_clean[col].mode().iloc[0]
    median_dict[col] = df_clean[col].median()
    variance_dict[col] = df_clean[col].var()
    std_sample_dict[col] = df_clean[col].std(ddof=1)
    std_population_dict[col] = df_clean[col].std(ddof=0)
    min_dict[col] = df_clean[col].min()
    max_dict[col] = df_clean[col].max()
    range_dict[col] = df_clean[col].max() - df_clean[col].min()
    q1_dict[col] = df_clean[col].quantile(0.25)
    q2_dict[col] = df_clean[col].quantile(0.50)
    q3_dict[col] = df_clean[col].quantile(0.75)
    iqr_dict[col] = df_clean[col].quantile(0.75) - df_clean[col].quantile(0.25) 

# Membuat ringkasan untuk DataFrame
df_clean_summary_iterative = pd.DataFrame({
    'Metric': columns_of_interest,
    'mean': list(mean_dict.values()),
    'mode': list(mode_dict.values()),
    'median': list(median_dict.values()),
    'variance': list(variance_dict.values()),
    'std_sample': list(std_sample_dict.values()),
    'std_population': list(std_population_dict.values()),
    'min': list(min_dict.values()),
    'Q1 (25% Percentile)': list(q1_dict.values()),
    'Q2 (50% Percentile)': list(q2_dict.values()),
    'Q3 (75% Percentile)': list(q3_dict.values()),
    'max': list(max_dict.values()),
    'range': list(range_dict.values()),  
    'IQR': list(iqr_dict.values())
})

# Membulatkan kolom angka menjadi 2 tempat desimal
df_clean_summary_iterative = df_clean_summary_iterative.round(2)

# Memunculkan dataframe dari kalkulasi
df_clean_summary_iterative

,Metric,mean,mode,median,variance,std_sample,std_population,min,Q1 (25% Percentile),Q2 (50% Percentile),Q3 (75% Percentile),max,range,IQR
0,quantity,11.95,1.00,4.00,4693.02,68.51,68.51,1.0,2.00,4.00,12.00,10000.00,9999.00,10.00
1,price,3.67,1.25,1.95,4953.48,70.38,70.38,0.0,1.25,1.95,3.75,25111.09,25111.09,2.50
2,total_amt,20.47,15.00,10.40,9552.36,97.74,97.74,0.0,4.25,10.40,17.70,25111.09,25111.09,13.45


> Mengubah urutan dari hasil output dari code diatas (_Transpose_):


In [33]:
df_clean_summary_itv_transposed = df_clean_summary_iterative.T
df_clean_summary_itv_transposed.columns = df_clean_summary_itv_transposed.iloc[0]
df_clean_summary_itv_transposed = df_clean_summary_itv_transposed[1:]
df_clean_summary_itv_transposed

Metric,quantity,price,total_amt
mean,11.95,3.67,20.47
mode,1.0,1.25,15.0
median,4.0,1.95,10.4
variance,4693.02,4953.48,9552.36
std_sample,68.51,70.38,97.74
std_population,68.51,70.38,97.74
min,1.0,0.0,0.0
Q1 (25% Percentile),2.0,1.25,4.25
Q2 (50% Percentile),4.0,1.95,10.4
Q3 (75% Percentile),12.0,3.75,17.7


> Membuat fungsi untuk menghasilkan output tanpa harus mentranspose:


In [34]:
# Mendefinisikan kolom untuk mengkalkulasi ringkasan statistikDefine the columns for which you want to calculate summary statistics
columns_to_analyze = ['quantity', 'price', 'total_amt']

# Menginisialisai kamus kosong untuk menyetorkan hasil
summary_stats = {}

# Perulangan tiap kolom
for col in columns_to_analyze:
    summary_stats[col] = {
        'mode': df_clean[col].mode().iloc[0],
        'median': df_clean[col].median(),
        'mean': df_clean[col].mean(),
        'variance': df_clean[col].var(),
        'std_sample': df_clean[col].std(ddof=1),
        'std_population': df_clean[col].std(ddof=0),
        'min': df_clean[col].min(),
        'Q1 (25% Percentile)': df_clean[col].quantile(0.25),
        'Q2 (50% Percentile)': df_clean[col].quantile(0.50),
        'Q3 (75% Percentile)': df_clean[col].quantile(0.75),  
        'max': df_clean[col].max(),
        'range': df_clean[col].max() - df_clean[col].min(),
        'IQR': df_clean[col].quantile(0.75) - df_clean[col].quantile(0.50)
    }

# Membuat dataframe baru
df_clean_stats_iterative = pd.DataFrame(summary_stats)

# Membulatkan desimal ke dua digit setelah tanda pemisah
df_clean_stats_iterative = df_clean_stats_iterative.round(2)

# Memunculkan dataframe
df_clean_stats_iterative

,quantity,price,total_amt
mode,1.00,1.25,15.00
median,4.00,1.95,10.40
mean,11.95,3.67,20.47
variance,4693.02,4953.48,9552.36
std_sample,68.51,70.38,97.74
std_population,68.51,70.38,97.74
min,1.00,0.00,0.00
Q1 (25% Percentile),2.00,1.25,4.25
Q2 (50% Percentile),4.00,1.95,10.40
Q3 (75% Percentile),12.00,3.75,17.70


### **4.2. Mendapatkan Statistik Deskriptif dengan Pengelompokkan Data**


#### **4.2.1. Statistik Deskriptif untuk agregasi dari satu pengelompokkan data pada suatu kolom**

Sebagai contoh untuk membuat agregasi berdasarkan filter nama produk dan kuantitas:


In [35]:
tea_agg1 = (
  df_clean[df_clean['product_name'].str.contains('tea')]
  .groupby('product_name',observed=True)['quantity']
  .sum()
  .sort_values(ascending=False)
)
tea_agg1

product_name
60 teatime fairy cake cases            27047
antique silver tea glass etched        14392
moroccan tea glass                      7019
antique silver tea glass engraved       6536
potting shed tea mug                    6088
                                       ...  
cubic mug teatime cakes                    6
english rose tea set in gift box           5
teatime funky flower backpack for 2        4
ceramic cake teapot with cherry            3
teatime round cake tins                    1
Name: quantity, Length: 85, dtype: int32

In [36]:
tea_agg = (
  df_clean[df_clean['product_name'].str.contains('tea')]
  .groupby(by='product_name',observed=True,as_index=True)
  .agg(total_quantity=pd.NamedAgg(column='quantity', aggfunc='sum'))
  .sort_values(by='total_quantity',ascending=False)
)
tea_agg = (
  tea_agg
  .reset_index()
  .rename(columns={'total_quantity':'sum_quantity'})
)
tea_agg

,product_name,sum_quantity
0,60 teatime fairy cake cases,27047
1,antique silver tea glass etched,14392
2,moroccan tea glass,7019
3,antique silver tea glass engraved,6536
4,potting shed tea mug,6088
...,...,...
80,cubic mug teatime cakes,6
81,english rose tea set in gift box,5
82,teatime funky flower backpack for 2,4
83,ceramic cake teapot with cherry,3


#### **4.2.2. Statistik Deskriptif untuk agregasi lebih dari pengelompokkan data lebih dari satu kolom**


Sebagai contoh untuk agregasi pengelompokkan data berdasarkan lebih dari satu kolom:


In [37]:
tea_order_status_agg = (
  df_clean[df_clean['product_name'].str.contains('tea')]
  .groupby(['product_name','order_status'],observed=True,as_index=True)
  .agg(total_quantity=pd.NamedAgg(column='quantity', aggfunc='sum'))
  .sort_values(by='total_quantity', ascending=False)
)
tea_order_status_agg = tea_order_status_agg.reset_index()
tea_order_status_agg

,product_name,order_status,total_quantity
0,60 teatime fairy cake cases,delivered,26895
1,antique silver tea glass etched,delivered,14285
2,moroccan tea glass,delivered,6930
3,antique silver tea glass engraved,delivered,6530
4,potting shed tea mug,delivered,5979
...,...,...,...
132,teatime round cake tins,cancelled,1
133,set 2 tea towels i love london,cancelled,1
134,tea time mug in gift box,cancelled,1
135,tea time breakfast basket,cancelled,1


#### **4.2.3. Statistik Deskriptif untuk beberapa agregasi tertentu dari beberapa Kolom**


Sebagai contoh untuk mendapatkan hasil beberapa agregasi berdasarkan pengelompokkan pada satu kolom:


In [38]:
df_tea_agg2 = (
  df_clean[df_clean['product_name'].str.contains('tea')]
  .groupby('product_name',observed=True, as_index=True)
  .agg(total_quantity=pd.NamedAgg(column='quantity',aggfunc='sum'),total_amount=('total_amt','sum'))
  .sort_values('total_amount', ascending=False)
)
df_tea_agg2

,total_quantity,total_amount
product_name,,
retro spot tea set ceramic 11 pc,5017,23075.27
tea time cake stand in gift box,1801,17851.95
red retrospot traditional teapot,2521,17719.99
antique silver tea glass etched,14392,16713.20
hot water bottle tea and sympathy,4083,15244.35
...,...,...
cubic mug teatime cakes,6,15.30
teatime pencil w stamp asst,36,15.12
ceramic cake teapot with cherry,3,13.45


Sebagai contoh untuk mencari beberapa aggregasi tertentu berdasarkan pengelompokkan dari beberapa kolom. Pada kasus dibawah dataframe merupakan product_name yang memiliki string 'tea':


In [39]:
df_tea_agg3 = (
  df_clean[df_clean['product_name'].str.contains('tea')]
  .groupby(['product_name','order_status'],observed=True, as_index=True)
  .agg(total_quantity=pd.NamedAgg(column='quantity',aggfunc='sum'),total_amount=('total_amt','sum'))
  .sort_values('total_amount', ascending=False)
)
df_tea_agg3

,,total_quantity,total_amount
product_name,order_status,,
retro spot tea set ceramic 11 pc,delivered,4615,21345.07
tea time cake stand in gift box,delivered,1789,17724.55
red retrospot traditional teapot,delivered,2420,16946.04
antique silver tea glass etched,delivered,14285,16593.13
hot water bottle tea and sympathy,delivered,3945,14759.25
...,...,...,...
ceramic cake teapot with cherry,delivered,1,4.95
set 2 tea towels i love london,cancelled,1,2.95
tea time mug in gift box,cancelled,1,2.95


### **4.3. Menangani Data Kategori**


#### **4.3.1. Mencari nilai unik untuk data kategori atau yang diberikan label**


> Mencari nilai unik dari suatu kolom dengan fungsi <code>.unique()</code>


In [40]:
df_clean['order_status'].unique()

array(['cancelled', 'delivered'], dtype=object)

> Mengagregasi nilai unik dari suatu kolom dengan fungsi <code>.value_counts()</code>


In [41]:
df_clean['order_status'].value_counts()

order_status
delivered    352892
cancelled      7936
Name: count, dtype: int64

In [42]:
# Mengkalkulasi perhitungan nilai untuk nilai unik 'order_status'
order_status_counts = df_clean['order_status'].value_counts()
order_status_df = pd.DataFrame(order_status_counts)
order_status_df

,count
order_status,
delivered,352892
cancelled,7936


#### **4.3.2. Mencari hasil agregasi dari pengelompokkan data**


Pada dasarnya fungsi ini seperti membuat pivot tabel di dalam excel untuk menangani Data dengan Kategori.

Fungsi dibawah membuat perhitungan penjumlahan terhadap kolom kuantitas berdasarkan pengelompokkan dari kolom order_status:


In [43]:
df_group = (
   df_clean
   .groupby(['order_status'], observed=True, as_index=True)['quantity']
   .sum()
   .sort_values(ascending=False)
)
df_group

order_status
delivered    4214751
cancelled      98822
Name: quantity, dtype: int32

Memunculkan dataframe dari hasil pengelompokkan dan agregasi satu kolom:


In [44]:
df_group1 = (
  df_clean
  .groupby(['order_status'], observed=True, as_index=True)
  .agg(sum_quantity=pd.NamedAgg(column='quantity',aggfunc='sum'))
  .sort_values(by='sum_quantity',ascending=False)
)
df_group1

,sum_quantity
order_status,
delivered,4214751
cancelled,98822


Memunculkan dataframe dari pengelompokkan berdasarkan beberapa kolom dan hasil dari beberapa agregasi:


In [45]:
df_group2 = (
  df_clean
  .groupby(['product_code','product_name','order_status'], observed=True,as_index=True)
  .agg(total_quantitiy=('quantity','sum'),total_amount=('total_amt','sum'))
  .sort_values('total_amount', ascending=False)
)
df_group2

,,,total_quantitiy,total_amount
product_code,product_name,order_status,,
M,manual,cancelled,1260,169753.910
22423,regency cakestand 3 tier,delivered,11398,131306.700
85123A,white hanging heart t-light holder,delivered,47727,127510.450
85099B,jumbo bag red retrospot,delivered,40866,71624.700
84879,assorted colour bird ornament,delivered,37667,59731.550
...,...,...,...,...
51020B,stripy design shower cap,cancelled,1,0.290
72708,sculpted round ivory candle,delivered,1,0.210
84201C,happy birthday card teddy/cake,delivered,1,0.190


Membuat fungsi yang mirip dengan pivot table, untuk demonstrasi:


In [46]:
import pandas as pd

# Sample data (replace this with your actual data)
data = {
    'Date': ['2024-08-01', '2024-08-01', '2024-08-02', '2024-08-02'],
    'Product': ['A', 'B', 'A', 'B'],
    'Sales': [100, 200, 150, 180]
}

df = pd.DataFrame(data)

# Initialize an empty dictionary to store aggregated results
pivot_dict = {}

# Specify the columns for the pivot table
index_columns = ['Date']
column_column = 'Product'
value_column = 'Sales'

# Iterate over unique values in the index column(s)
for index_value in df[index_columns[0]].unique():
    filtered_df = df[df[index_columns[0]] == index_value]
    pivot_partial = filtered_df.pivot_table(index=[], columns=column_column, values=value_column, aggfunc='sum')
    pivot_dict[index_value] = pivot_partial

# Concatenate the partial pivot tables into the final result
final_pivot_table = pd.concat(pivot_dict.values(), keys=pivot_dict.keys())

final_pivot_table

,Product,A,B
2024-08-01,Sales,100,200
2024-08-02,Sales,150,180
